## <b> Modeling Insights </b> notebook (Nr 4) for <b>Project 2</b> of General Assembly DSIR-0320 
by Martijn de Vries </br>
martijndevries91@gmail.com

## Problem Statement

A real estate company in Ames, Iowa is looking for a new way to evaluate the market value of a house. Using the Ames data set as training data, we will build a predictive linear regression model to predict the sale price of a house as well as possible.

To gauge the model performance, I will compare my results against a 'benchmark model', which is a simple OLS regression of total living area versus sale price. I will try out different models with different numbers of features using different linear regression techniques, and ultimately identify which model does the best job at predicting the market value of the house.

## In this notebook

In the previous Modeling notebook I fit a bunch of different models using different strategies and regressors. In this notebook, I want to hone in on the best model, compare it to the benchmark model, and figure out why this model worked the best

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

#sklearn imports
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.pipeline import make_pipeline

from statsmodels.tools import tools
import statsmodels.api as sm

In the previous notebook we saw that the most complex model, model 4, seems to give the best predictions - for both an OLS regression and Ridge Regression (and when combined with a log-transform of the Sale Price). Let's refit the model using statsmodels and use the summary() method to evaluate the P-value scores of different parameters

In [17]:
def print_metrics(r_obj, X_train, X_val, y_train, y_val, islog=False):
    """
    for a given regression object, and input training/validation data, print out 1) the r2 score, 2) the cross-validation score of the training data, and 
    3) the RMSE for both the training and validation data
    assumes y-values are not log transformed. if islog=True, the log transformation happens within this function
    """
    
    if islog == False:
        train_score = r_obj.score(X_train, y_train)
        val_score = r_obj.score(X_val, y_val)
    elif islog == True:
        train_score = r_obj.score(X_train, np.log(y_train))
        val_score = r_obj.score(X_val, np.log(y_val))
    print(f'The training r2 score is {str(train_score)}')
    print(f'The validation r2 score is {str(val_score)}')

    if islog == False:
        crossval = np.average(cross_val_score(r_obj, X_train, y_train))
    elif islog == True:
        crossval = np.average(cross_val_score(r_obj, X_train, np.log(y_train)))
    
    print(f'The cross validation score is {str(crossval)}')
    print('-'*60)
                
    preds_train = r_obj.predict(X_train)
    preds_val = r_obj.predict(X_val)
    
    #RMSE in dollars
    if islog == False:
        rmse_val =  mean_squared_error(y_val, preds_val, squared=False)
        rmse_train =  mean_squared_error(y_train, preds_train, squared=False)
    elif islog == True:
        rmse_val =  mean_squared_error(y_val, np.exp(preds_val), squared=False)
        rmse_train =  mean_squared_error(y_train, np.exp(preds_train), squared=False)
        
    print(f'The training RMSE score is {str(rmse_train)}')
    print(f'The validation RMSE score is {str(rmse_val)}')
    
    return

In [18]:
#load in the feature engineered training data for model 4
#this model 
df_model4= pd.read_csv('../model_inputs/train_engineered_m4.csv')
df_model4.head()

,Id,tot_area,Garage Area,Lot Frontage,yr_built,yr_remod,qual,tot_rooms_abv,full_bath,mas_vnr_area,...,Bsmt Qual_Fa,Bsmt Qual_Gd,Bsmt Qual_NP,Bsmt Qual_Po,Central Air_N,Alley_Grvl,Alley_Pave,Paved Drive_N,Paved Drive_P,SalePrice
0,109,2204.0,475.0,89.778892,1976,2005,6,6,2,289.0,...,0,0,0,0,0,0,0,0,0,130500
1,544,3035.0,559.0,43.000000,1996,1997,7,8,2,132.0,...,0,1,0,0,0,0,0,0,0,220000
2,153,2114.0,246.0,68.000000,1953,2007,5,5,1,0.0,...,0,0,0,0,0,0,0,0,0,109000
3,318,1828.0,400.0,73.000000,2006,2007,5,7,2,0.0,...,0,1,0,0,0,0,0,0,0,174000
4,255,2121.0,484.0,82.000000,1900,1993,6,6,2,0.0,...,1,0,0,0,0,0,0,1,0,138500


In [19]:
X = df_model4.drop(columns=['Id', 'SalePrice'])
y = df_model4['SalePrice']
X_train, X_val, y_train, y_val= train_test_split(X, y, random_state=42)

In [20]:
#Fit with sklearn
lr = LinearRegression()
lr.fit(X_train,np.log(y_train))
print_metrics(lr, X_train, X_val, y_train, y_val,islog=True)

The training r2 score is 0.9205086047396134
The validation r2 score is 0.8830169861886302
The cross validation score is 0.908501590825022
------------------------------------------------------------
The training RMSE score is 21674.95646074773
The validation RMSE score is 22312.439659127


In [21]:
X_train_c = tools.add_constant(X_train)
model = sm.OLS(np.log(y_train),X_train_c).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.921
Model:                            OLS   Adj. R-squared:                  0.917
Method:                 Least Squares   F-statistic:                     249.7
Date:                Tue, 11 Apr 2023   Prob (F-statistic):               0.00
Time:                        12:58:59   Log-Likelihood:                 1149.9
No. Observations:                1535   AIC:                            -2162.
Df Residuals:                    1466   BIC:                            -1794.
Df Model:                          68                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    4.6283      0.647      7.154      0.000       3.359       5.897
tot_area                 0.0002   8.57e-06     22.729      0.000       0.000       0.000
Garage Area              0.0001   3.59e-05      4.109      0.000    7.71e-05       0.000
Lot Frontage             0.0006      0.000      3.763      0.000       0.000       0.001
yr_built                 0.0021      0.000      7.381      0.000       0.002       0.003
yr_remod                 0.0010      0.000      3.977      0.000       0.000       0.001
qual                     0.0538      0.004     12.447      0.000       0.045       0.062
tot_rooms_abv            0.0110      0.003      3.688      0.000       0.005       0.017
full_bath                0.0018      0.008      0.206      0.837      -0.015       0.018
mas_vnr_area          2.417e-05   2.17e-05      1.114      0.266   -1.84e-05    6.68e-05
fireplaces               0.0550      0.006      9.119      0.000       0.043       0.067
cond                     0.0458      0.004     12.003      0.000       0.038       0.053
Garage Cars              0.0053      0.010      0.514      0.607      -0.015       0.025
cond_RRAe               -0.0813      0.029     -2.817      0.005      -0.138      -0.025
cond_PosA                0.0627      0.038      1.634      0.102      -0.013       0.138
cond_Artery             -0.0417      0.018     -2.379      0.018      -0.076      -0.007
cond_Feedr              -0.0350      0.015     -2.409      0.016      -0.064      -0.007
cond_PosN                0.0326      0.029      1.141      0.254      -0.023       0.089
cond_RRAn                0.0226      0.023      0.986      0.324      -0.022       0.068
cond_RRNe               -0.0917      0.084     -1.088      0.277      -0.257       0.074
cond_RRNn               -0.0280      0.054     -0.520      0.603      -0.134       0.078
MS Zoning_C (all)       -0.2481      0.041     -6.051      0.000      -0.328      -0.168
MS Zoning_FV             0.0747      0.028      2.669      0.008       0.020       0.130
MS Zoning_I (all)       -0.1008      0.123     -0.821      0.412      -0.342       0.140
MS Zoning_RH             0.0419      0.040      1.041      0.298      -0.037       0.121
MS Zoning_RM            -0.0418      0.016     -2.546      0.011      -0.074      -0.010
Func_Maj                -0.1805      0.033     -5.456      0.000      -0.245      -0.116
Func_Min                -0.0070      0.016     -0.433      0.665      -0.039       0.025
Func_Mod                -0.0596      0.027     -2.217      0.027      -0.112      -0.007
Func_Sev                -0.4492      0.085     -5.271      0.000      -0.616      -0.282
Neighborhood_Blmngtn    -0.0445      0.033     -1.343      0.179      -0.110       0.021
Neighborhood_Blueste    -0.0727      0.052     -1.395    

It seems that the numerical columns (ie not categorical dummy columns) with the highest P-values (that is, the probability that these parameters are just fit to noise) are Garage Cars and Full Bath. Will the model get better if I get rid of those?

In [22]:
X = df_model4.drop(columns=['Id', 'SalePrice', 'full_bath', 'Garage Cars'])
y = df_model4['SalePrice']
X_train, X_val, y_train, y_val= train_test_split(X, y, random_state=42)

In [23]:
lr = LinearRegression()
lr.fit(X_train,np.log(y_train))

LinearRegression()

In [24]:
print_metrics(lr, X_train, X_val, y_train, y_val,islog=True)

The training r2 score is 0.9204911110079034
The validation r2 score is 0.8827456064942821
The cross validation score is 0.9085772206180296
------------------------------------------------------------
The training RMSE score is 21673.07742284863
The validation RMSE score is 22314.573728817486


It seems the model is virtually the same, maybe slightly worse. So that doesn't really help.

Overall, from the modeling notebook and the Kaggle website, it seems that the best performing model is 

## Comparison of model to the benchmark

As the benchmark model, we set a simple OLS regression model of total living area to sale price. Looking at the metrics of this model again:

In [27]:
X_b = df_model4[['tot_area']]
y_b = df_model4['SalePrice']
X_train_b, X_val_b, y_train_b, y_val_b= train_test_split(X_b, y_b, random_state=42)
lr_b = LinearRegression()
lr_b.fit(X_train_b,y_train_b)

print_metrics(lr, X_train_b, X_val_b, y_train_b, y_val_b,islog=False)

The training r2 score is 0.6943023042761038
The validation r2 score is 0.6387963066523947
The cross validation score is 0.6918200458030501
------------------------------------------------------------
The training RMSE score is 43672.87991366827
The validation RMSE score is 47900.07573530223
